In [1]:
import pandas as pd
import numpy as np
excelFile = r'files/sw_train_data_sample.xlsx'
sample_df = pd.DataFrame(pd.read_excel(excelFile))
print(f"{sample_df.shape}")
sample_df.head()

(348, 9)


,code,name,kPattern,buy_price,sell_price,k,j,label_sell_price,label_buy_price
0,801881,其他交运设备II,712,-1.098986,-0.008333,95.583908,90.605103,0.290301,-1.715114
1,801881,其他交运设备II,712,-0.511574,0.225401,70.570293,59.899220,2.299587,0.042024
2,801881,其他交运设备II,712,2.521705,5.227228,67.502247,54.059976,8.238283,5.253366
3,801881,其他交运设备II,712,-2.102709,-0.082342,83.430366,84.083528,3.585147,-0.389238
4,801881,其他交运设备II,712,-0.882167,-0.200809,40.766058,24.982887,1.656360,-1.291246


In [2]:
def set_0_or_1(x):
    if x >= 2:
        return 1
    return 0

def percent_to_one(x):
    return int(x * 100) /1000.0

def to_one_min_max_120(x):
    v = x / 120
    if  v > 1:
        v = 1
    if  v< -1:
        v =-1
    return int(v*100)/100.0

def toInt(x):
    return int(x + 0.5)

d = sample_df
d['buy_price'] = d.buy_price.apply(percent_to_one)   #归一化
d['sell_price'] = d.sell_price.apply(percent_to_one)   #归一化
d['label_sell_price'] = d.label_sell_price.apply(set_0_or_1)
d['label_buy_price'] = d.label_buy_price.apply(toInt)
d.k = d.k / 100
d.j = d.j / 100
sample_df = sample_df.drop(columns=['code','kPattern', 'name'])
#sample_df.rename(columns={'d':'kd_diff'},inplace=True)
sample_df.head()
 

,buy_price,sell_price,k,j,label_sell_price,label_buy_price
0,-0.109,0.000,0.955839,0.906051,0,-1
1,-0.051,0.022,0.705703,0.598992,1,0
2,0.252,0.522,0.675022,0.540600,1,5
3,-0.210,-0.008,0.834304,0.840835,1,0
4,-0.088,-0.020,0.407661,0.249829,0,0


In [4]:
import sklearn
from sklearn import model_selection

data  = sample_df.values
x,y=np.split(data,indices_or_sections=(4,),axis=1) #x为数据，y为标签
y = y[:,0:1].astype('int')  #取第一列

In [5]:
train_data,test_data,train_label,test_label =model_selection.train_test_split(x,y, train_size=0.7,test_size=0.3)
train_label = train_label.ravel()
test_label = test_label.ravel()

In [13]:
from sklearn import svm
classifier=svm.SVC(C=2,kernel='rbf',gamma=10,decision_function_shape='ovr',probability=True) # ovr:一对多策略
classifier.fit(train_data,train_label) #ravel函数在降维时默认是行序优先

print("训练集：",classifier.score(train_data,train_label))
print("测试集：",classifier.score(test_data,test_label))

训练集： 0.7818930041152263
测试集： 0.8380952380952381


In [10]:
sampel_1 = test_data[5:10,]   ##取测试数据低第8,9个
sampel_1

array([[-0.043     , -0.003     ,  0.93704555,  0.88734649],
       [ 0.001     ,  0.076     ,  0.68050659,  0.6686651 ],
       [-0.038     ,  0.029     ,  0.27339842,  0.20495555],
       [-0.018     ,  0.028     ,  0.66045136,  0.62458342],
       [-0.087     ,  0.006     ,  0.59271642,  0.56355075]])

In [14]:
predict_props = classifier.predict_proba(sampel_1)
predict_label = classifier.predict(sampel_1)
predict_label,predict_props

(array([0, 0, 0, 0, 0]),
 array([[0.83177423, 0.16822577],
        [0.57900388, 0.42099612],
        [0.64109875, 0.35890125],
        [0.72452047, 0.27547953],
        [0.64957878, 0.35042122]]))

,kPattern,k_diff,d,macd,open,short,long
0,46418,76.080320,18.173668,-4.023488,-0.034919,0.467475,0.033330
1,39776,82.990059,9.706241,-2.389732,-0.020127,0.237116,-0.194516
2,33215,86.664484,4.752863,-0.932882,-0.014234,0.127453,-0.193571
3,33215,91.173888,4.231078,0.055731,0.052279,1.463844,0.923712
4,33216,93.380019,2.973889,1.164693,-0.038353,0.239515,-0.158650


In [20]:
d = sample_df
sample_df['d'] = d.k - d.d
sample_df.head()

,code,name,kPattern,k,d,dif,dea,macd,open,short,long
0,801012,农产品加工II,46418,76.080320,18.173668,11.395251,15.418739,-4.023488,-0.349192,0.467475,0.033330
1,801012,农产品加工II,39776,82.990059,9.706241,11.625503,14.015235,-2.389732,-0.201269,0.237116,-0.194516
2,801012,农产品加工II,33215,86.664484,4.752863,11.312657,12.245539,-0.932882,-0.142343,0.127453,-0.193571
3,801012,农产品加工II,33215,91.173888,4.231078,11.253561,11.197830,0.055731,0.522793,1.463844,0.923712
4,801012,农产品加工II,33216,93.380019,2.973889,12.710930,11.546237,1.164693,-0.383528,0.239515,-0.158650
